In [43]:
import numpy as np
import pandas as pd

In [44]:
def preprocess(df):
    from sklearn.preprocessing import OrdinalEncoder
    from sklearn.pipeline import Pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn.impute import SimpleImputer
    pipe_enp = Pipeline([('imp', SimpleImputer(strategy='most_frequent')),
                        ('enc', OrdinalEncoder())])
    
    df['nFam'] = df['SibSp'].copy() + df['Parch'].copy()
    df['Tag'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
    df['Cabin'] = df.Cabin.apply(lambda x: x[0] if pd.notna(x) else 'NAN')
    df['Sex'] = df['Sex'].apply(lambda x: 1 if x=='female' else 0)
    df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket'], inplace=True, axis=1),
                     
                     

    pipe_imp_age = SimpleImputer()

    df[['Embarked', 'Tag', 'Cabin']] = pipe_enp.fit_transform(np.c_[df[['Embarked', 'Tag', 'Cabin']]])


    df[['Age', 'Fare']] = pipe_imp_age.fit_transform(np.c_[df[['Age', 'Fare']]])
    Cat_cols = ['Sex', 'Pclass', 'Cabin', 'Embarked', 'Tag' ]
    CounN_cols = ['Age', 'Fare', 'nFam']
    df.Fare = df.Fare.apply(lambda x: df.Fare.mean() if x == 0 else x)
    features, labels = df.iloc[:, 1:], np.c_[df.iloc[:, 0]]
    features = np.c_[(features - features.mean()) / features.std()]
    return features, labels

In [45]:
from sklearn.datasets import openml
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
titanic = pd.read_csv('train.csv') #Load the data
X, y = preprocess(titanic)#it imputes, encodes and scales our dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2) #split the data 


<center><h3><b>Random Forest Classifier Big Picture & Technical stuff:</b></h3></center>

<b>Introduction : </b>
	First of all we gonna introduce the Big Picture of Decision Trees which is the learning technique that Random Forest built upon and we gonna dive in on how the Decision Tree really work after that we gonna show how Random Forest works.  Don't worry there is not a lot of math evolved there is just little bit of multplication and summation :)) so enjoy reading!<br><br>
<b>Big Picture : </b>
        Say you wanna make a function (algorithm) that takes an information about a fruit (color, weight, shape) and outputs whether the given sample (color, weight, shape) is a banana or a red apple the algorithm needs to find that the best way to differ between a banana and a red apple is the color (the shape too but let's pretend that the color is the best feature to separate between these two labels ) so the algorithm should learn that if a given sample has color equals to yellow then it's a banana else it's a red apple, say now we have 3 kind of labels red apples, oranges and bananas doing the same approach it will find that if the color of the sample is yellow then it's banana now we took care about how to label a sample to the banana class then it moves to how to separate the remaining classes and it will find that if color of the sample is red then it's apple else orange(or if the color of the sample is orange then it's an orange else it's an apple), okay that was simple and we could just type that using an if statements but imagine if you had 1000 type of fruits, probably you'll get bored typing all of these if statments right? and that would be hard too! that's were decision tree comes in.
<img src="dct.jpg" width="750" height="500" align="center"/>
<h5><b>The Below Picture shows some important Terminology </b></h5>
<img src="Terminology.png" width="750" height="500" align="center"/><br><br>
Decision Trees works as follows it finds the best feature (threshold's column) and it's specific value (threshold) that best separate the classes (that's the root node which is the first decision node that separates the dataset) now we end up having dataset that have value below a certain specific value (threshold) in that feature (threshold's column) and dataset that have value above a certain specific value (threshold) in that feature (threshold's column) now for each of the two datasets check if they are pure meaning they have only one class in them say the dataset that was above threshold was pure and it had only the class A it then we save that and we say for each new sample if it had in the threshold's column value above the threshold then it's belongs to the class A (that's the Leaf node), back to our two new datasets (dataset below/above threshold) if one of these datasets or both wasn't pure we repeat the process on the dataset/s that wasn't pure, for example data below threshold wasn't pure then we take it we find the best threshold that separate the classes in it then get our datasets that above and below threshold checks wether they were pure if indeed they were pure then we produce 2 leaf nodes(one for the class that the dataset above the last threshold have and the other for the class that the dataset below the last threshold have), if the two or one of them wasn't pure we repeat until we get pure datasets

<b>Being Technical</b> : In this part we will illustrate what every method (function) of the class DecisionTreeClassifier does then we will show how they all work together in a smooth way to create our Potential random Forest:

<h5><b> DecisionTreeClassifier's methods illustrated : <b></h5><br>
{'function_name' : ispure, 'what_it_does': 'it checks wether a given array contains a single class (True) or if it has more than 1 class (False)', 'examples': ispure(np.array([1, 1, 1, 1])) returns True ,ispure(np.array([1, 2, 5, 2])) returns False}<br><br>
{'function_name': most_frequent, 'what_it_does':'it returns the most frequent value in the given array', 'example':most_frequent(np.array([25, 6, 2, 3, 2, 5, 4, 2])) returns 2}<br><br>
{'function_name': impurity_measure, 'what_it_does': 'basically you can think of it as a measure of how different the values are in the array where using entropy equation calculates how random the values in the array are and gini calculates how random the values in the array would be if they were sampled from random distribution', 'example': impurity_measure(np.array([2, 2, 2, 2, 2])) whether we used entropy or gini equation it will return 0 since the values in the array are not different and if it returned value > 0 in another example that means the values of the given array are different}<br><br>
{'function_name': overall_impurity, 'what_it_does': it calculates how well a given threshold separates the data by giving it the labels of the samples that have a value above the threshold in the threshold's column and the labels of the samples that have a value below that threshold in the threshold's column, 'example': 'overall_impurity(np.array([0, 0, 0, 0, 0]), np.array([1, 1, 1, 1, 1])) it returns 0 because that threshold pefectly separates the two labels if returned value > 0 in another case that means that the threshold doesn't separate the labels perfectly there is impurity'}<br><br>
{'function_name': all_thresholds, 'what_it_does' :'giving it a numpy 2D array for each column it does (ARRAY[row, column]+ARRAY[row-1, column])/2 with for each row > 0 that means it calculates the potential thresholds that could separate the dataset's classes, 'example':'say this is your values in the first column [1, 2, 3, 4] it makes a key in the dictinary has the index of that column and its potential thresholds for example for the fisrt column the dictinary would be {0: [(1+2)/2, (2+3)/2, (3+4)/2]} which equals to {0:[1.5, 2.5, 3.5]}'}<br><br>
{'function_name': calculate_best_threshold, 'what_it_does':'from the potential thresholds it calculates the best threshold  that separates the classes in the current dataset'}<br><br>
{'function_name': create_tree, 'what_it_does': 'it creates our beloved Decision Tree, it's a recursive function and the best way to understand it is to go through it, max_depth argument stops the the decision from growing when it reaches that depth, min_samples_split is removing potential sub-tree and replace it with the most frequent class in that dataset when the length of the dataset reachs that min_samples_split and these argument are just regularizations to avoid overfitting'}<br><br>
{'function_name': split_data_using_threshold, 'what_it_does': 'returns the data below the threshold in the threshold's column and the data above the threshold in the threshold's column'},
{'function_name': predict_sample, 'what_it_does': 'returns the prediction of the giving sample, it's recursive function too'},
{'function_name': predict, 'what_it_does': 'returns the prediction of the giving samples/sample'},
{'function_name': col_threshold, 'what_it_does': 'it helps us to get from the way the tree is stored the threshold and the column of the node that we are in to move to the next node (we use in the prediction part)'}
    

<br><br>
so how they all work together? alright focus with me now, we start by calculating the best threshold and it's column that best separates the data then we get the data that has value above the threshold in the threshold's column and the data that has value below the threshold in the threshold's column then we check wether the data_below_the_threshold or data_above_the_threshold is pure or satisfies one of the regularizations if yes we put as leaf the most frequent class in that dataset else the data_below/above is not pure so it goes in the create_tree and this whole process repeats until the two datasets make in a recursive iteration one/two of the regularizations satisfied or both of the datasets are pure, if you didn't understand that it's REALLY okay, become familiar with recursivity and then go through the create_tree using pen and paper and hopefully you'll understand it.<br><br>
<b>Notes : </b>
    - use the example of both data_below_threshold and data_above_threshold being pure from the first recursive iteration, doing that will help you understand the creation of the Decision Tree
    - Decision trees has low bias and high variance that means they don't care if the data was quadratic or linear or ... and high variance basically it means they overfit(most of the time), Linear Regression model has high bias and low variance since Linear regression expects the data to be linear and it will have low variance if the data was indeed linear
<h5><b> RandomForest's Big Picture :<b></h5><br>
    Random Forest is just a bunch of decision trees each tree has random samples associated with random columns from the dataset for example say our dataset has 1000 sample and 5 columns then our first decision tree may have 780  random samples and the columns 1 , 2 and 4, the second decision tree will have 780 random sample and the columns may be 1, 3 and 5 and so on ..., the number of Decision Trees in the Random Forest is controlled by the argument n_estimators, how many samples you want to train the Decision Tree on is controlled by the argument max_samples_ratio you give it value between (0..1] and it calculates how many samples it must give to the decision tree for example max_samples_ratio = 0.7 that means give the decision tree 70% of all the samples and how many column it must feed the tree is controlled by the max_features_ratio, for example if max_samples_ratio = 0.6 and max_features_ratio = 0.8 that means feed each Decision Tree random 60% of all the samples and random 80% of all the features (columns), and the way it classifies the class it runs the sample in each Decision Tree and saves the output then after going through each tree it outputs the most frequent predicted class for example say we have Random Forest with 10 trees we give each one of them the same sample, 7 trees predicts that this sample belongs to 1's class and the other 3 trees says that this sample belongs to the 2's class so the Random Forest outputs 1 since most of the trees predicted 1 for the sample.
<h5><b> But why is this approach is better than a simple Decision Tree?<b></h5>
By feeding random samples and columns from the dataset to the trees they will generalize in different ways and that's great that means each tree has it strength and it's weakness so if one made a mistake the other trees will help to make the correct classification, Nevertheless they make commun errors, by doing that it will solve a little bit of the high variance problem in the Decision Tree.
<center><h5><b>Random Forest's Error Space Expectation versus Reality</b></h5></center>
<img src="EvsR.jpg" width="750" height="500" align="center"/>

In [46]:
class DecisionTreeClassifier(object):
    def __init__(self, max_depth=6, min_samples_split=3, imp_measure='gini'):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.imp_measure = imp_measure
        self.tree = None
    
    def impurity_measure(self, y, imp_measure='gini'):
        nichts, data_points = np.unique(y, return_counts=True) 
        p = data_points / sum(data_points)
        if imp_measure == 'entropy':
            return sum(-p*np.log2(p))
        elif imp_measure == 'gini':
            return 1-sum(np.square(p))
    
    def calculate_best_threshold(self, X, y, impurity_measure='gini'):
        X = np.c_[X]
        all_threshs = self.all_thresholds(X)
        maxi = np.inf
        best_threshold = 0
        its_column = 0
        for key in all_threshs.keys():
            for threshold in all_threshs[key]:
                y_above, y_below = y[X[:, key] >= threshold ], y[X[:, key] < threshold ]
                overall_imp = self.overall_impurity(y_above, y_below, impurity_measure)
                if overall_imp <= maxi:
                    maxi = overall_imp
                    best_threshold = threshold
                    its_column = key
        return best_threshold, its_column
    
    def overall_impurity(self, y1, y2, imp_measure='gini'):
        len_y1,  len_y2 = len(y1), len(y2)
        len_data = len_y1+len_y2
        if imp_measure == 'entropy':
            return (len_y1/len_data)*self.impurity_measure(y1, 'entropy')+(len_y2/len_data)*self.impurity_measure(y2, 'entropy')
        elif imp_measure == 'gini':
            return (len_y1/len_data)*self.impurity_measure(y1, 'gini')+(len_y2/len_data)*self.impurity_measure(y2, 'gini')
        else:
            raise NameError('{} is not supported as impurity measure please use gini or entropy'.format(imp_measure))
    def most_frequent(self, y):
        unique_vals, counts_unique_vals = np.unique(y, return_counts=True) #in this case since we told it to return the counts meaning how many each unique value appeared in the array so it will return the unique values and their counts
        return unique_vals[np.argmax(counts_unique_vals)]
    
    def all_thresholds(self, X):
        D = {}
        for c in range(X.shape[1]):
            D[c] = []
            for l in range(X.shape[0]):
                if l > 0:
                    D[c].append((X[l-1, c] + X[l, c])/2)
        return D
        
    def ispure(self, y):
        unique_vals = np.unique(y)
        return len(unique_vals) == 1 

    def create_tree(self, X, y, counts=1, max_depth=-1, min_samples_split=-1):
        max_depth, min_samples_split=self.max_depth , self.min_samples_split
        best_threshold, its_column = self.calculate_best_threshold(X, y, impurity_measure=self.imp_measure)
        X_below, y_below, X_above, y_above = self.split_data_using_threshold(X, y, best_threshold, its_column)
        if len(y) >= min_samples_split and counts <= max_depth and len(y_above) != 0 and len(y_below) != 0:
            counts +=1
            if self.ispure(y_below) == False:
                yes = self.create_tree(X_below, y_below, counts=counts, max_depth=max_depth, min_samples_split=min_samples_split)
            else:
                yes = self.most_frequent(y_below)
            if self.ispure(y_above) == False:
                no = self.create_tree(X_above, y_above, counts=counts, max_depth=max_depth, min_samples_split=min_samples_split )
            else:
                no = self.most_frequent(y_above)
            return {str(its_column)+'_'+str(best_threshold) : {True:yes, False:no}}
        else:
            return {str(its_column)+'_'+str(best_threshold) : {True:self.most_frequent(y), False:self.most_frequent(y)}}
   
    def predict_sample(self, D, X):
        if 'int' in str(type(D)) or 'float' in str(type(D)):
            return D
        else:
            col, thresh = self.col_threshold(tuple(D.keys())[0])
            D = D[tuple(D.keys())[0]]
            D = D[X[col] < thresh]
            return self.predict_sample(D, X)
    
    def predict(self, X):
        X = np.c_[X] #making sure even if it's one sample it gets in the form of 2D
        y_pred = []
        for i in range(X.shape[0]):
            y_pred.append(self.predict_sample(self.tree, X[i]))
        return y_pred

    
    def train(self, X, y): #make sure X is a 2D array even if they were DataFrames or Series they r converted to 2D so the code workds
        X, y = np.c_[X], np.c_[y]
        self.tree = self.create_tree(X, y)
    
    col_threshold = lambda self, a: (int(a[:a.index('_')]), float(a[a.index('_')+1:]))
    split_data_using_threshold = lambda self, X, y, threshold, col : (X[X[:, col] < threshold], y[X[:, col] < threshold], X[X[:, col] >= threshold], y[X[:, col] >= threshold] )

In [47]:
tree = DecisionTreeClassifier()

In [48]:
tree.train(X_train, y_train)

In [49]:
print('Accuracy :',(tree.predict(X_test) == y_test.reshape(-1)).sum()/ X_test.shape[0])

Accuracy : 0.8100558659217877


In [50]:
class RandomForestClassifier(DecisionTreeClassifier):
    def __init__(self, max_depth=6, min_samples_split=3,  n_estimators=40, max_features_ratio=1, max_samples_ratio=1, imp_measure='gini', verbose=True):
        DecisionTreeClassifier.__init__(self, max_depth=max_depth, min_samples_split=min_samples_split, imp_measure=imp_measure)
        self.n_estimators = n_estimators
        self.max_features_ratio = max_features_ratio
        self.max_samples_ratio = max_samples_ratio
        self.verbose = verbose
        self.estimators = None
        self.selected_feature_per_estimator = None
    
    def select_data(self, X, y, n_samples, n_features):
        columns = []
        while len(columns) < n_features:
            r = np.random.randint(0, X.shape[1], 1)
            if r not in columns:
                columns.append(r)
        rows = np.random.randint(0, n_samples, n_samples)
        columns = np.array(columns).reshape(-1)
        X = X[:, columns]
        X = X[rows, :]
        y = y[rows]
        return X, y, columns
            
    def train(self, X, y):
        
        n_features = round(X.shape[1]* self.max_features_ratio)
        n_samples = round(X.shape[0]* self.max_samples_ratio)
        if self.max_features_ratio < 0 or self.max_features_ratio == 0 or self.max_features_ratio > 1:
            raise ValueError('max_features_ratio must be positif between (0..1]')
        elif self.max_samples_ratio < 0 or self.max_samples_ratio == 0 or self.max_samples_ratio > 1:  
            raise ValueError('max_samples_ratio must be positif between (0..1]')
        elif n_features == 0:
            raise ValueError('Number of selected features per tree == 0 try to augment the value of max_features_ratio')
        elif n_samples == 0:
            raise ValueError('Number of selected samples per tree == 0 try to augment the value of max_samples_ratio')
        
        estimators = []
        selected_features_per_estimator = []
        for i in range(self.n_estimators):
            data_x, data_y, selected_features = self.select_data(X, y, n_samples=n_samples, n_features=n_features)
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split, imp_measure=self.imp_measure )
            tree.train(data_x, data_y)
            estimators.append(tree)
            selected_features_per_estimator.append(selected_features)
            if self.verbose:
                print('Tree', i+1, 'Done Training!')
        self.estimators, self.selected_features_per_estimator = estimators, selected_features_per_estimator

    def predict(self, X):
        if self.estimators == None:
            raise Exception('You must train the model before making any prediction! use model.train(X, y)')
        else:
            y_pred = np.zeros((X.shape[0], len(self.estimators)))
            for i in range(len(self.estimators)):
                y_pred[:, i] = self.estimators[i].predict(X[:, self.selected_features_per_estimator[i]])
            preds = []            
            for row in y_pred:
                uniques, m_freq = np.unique(row, return_counts=True)
                preds.append(uniques[np.argmax(m_freq)])
            return preds
        
        
            

In [57]:
rfc = RandomForestClassifier(n_estimators=20, max_features_ratio=0.8, max_samples_ratio=0.5)

In [58]:
rfc.train(X_train, y_train)

Tree 1 Done Training!
Tree 2 Done Training!
Tree 3 Done Training!
Tree 4 Done Training!
Tree 5 Done Training!
Tree 6 Done Training!
Tree 7 Done Training!
Tree 8 Done Training!
Tree 9 Done Training!
Tree 10 Done Training!
Tree 11 Done Training!
Tree 12 Done Training!
Tree 13 Done Training!
Tree 14 Done Training!
Tree 15 Done Training!
Tree 16 Done Training!
Tree 17 Done Training!
Tree 18 Done Training!
Tree 19 Done Training!
Tree 20 Done Training!


In [59]:
print('Accuracy :',(rfc.predict(X_test) == y_test.reshape(-1)).sum()/ X_test.shape[0])

Accuracy : 0.8324022346368715


<b>Our beloved Random Forest did better that a Decision Tree by 2% Great!</b>